<a href="https://colab.research.google.com/github/docheem/NLP-Portfolio/blob/main/PR_Conv_Text_State_of_the_Art_Embeddings_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting Text to Features Using State-of-the-Art Embeddings

In [ ]:
#Importing modules
import pandas as pd
import numpy as np
import scipy
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer # used for preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
import spacy
from tqdm import tqdm
import re
import matplotlib.pyplot as plt # our main display package
import plotly.graph_objects as go
import tensorflow_hub as hub
import tensorflow as tf
print(tf.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


2.11.0


Establishing UTF-8

In [ ]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

#EDA

In [ ]:
import pandas as pd
import csv

path = '/content/drive/MyDrive/DAtasets/ted_main.csv'

ted_df = pd.read_csv(path)#, quoting=csv.QUOTE_NONE, sep=';', on_bad_lines='skip', encoding=None)
ted_df.head(3)



,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292


In [ ]:
ted_df['description']

0       Sir Ken Robinson makes an entertaining and pro...
1       With the same humor and humanity he exuded in ...
2       New York Times columnist David Pogue takes aim...
3       In an emotionally charged talk, MacArthur-winn...
4       You've never seen data presented like this. Wi...
                              ...                        
2545    Between 2008 and 2016, the United States depor...
2546    How can you study Mars without a spaceship? He...
2547    Science fiction visions of the future show us ...
2548    In an unmissable talk about race and politics ...
2549    With more than half of the world population li...
Name: description, Length: 2550, dtype: object

In [ ]:
ted_df.dtypes

comments               int64
description           object
duration               int64
event                 object
film_date              int64
languages              int64
main_speaker          object
name                  object
num_speaker            int64
published_date         int64
ratings               object
related_talks         object
speaker_occupation    object
tags                  object
title                 object
url                   object
views                  int64
dtype: object

In [ ]:
ted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2550 non-null   int64 
 1   description         2550 non-null   object
 2   duration            2550 non-null   int64 
 3   event               2550 non-null   object
 4   film_date           2550 non-null   int64 
 5   languages           2550 non-null   int64 
 6   main_speaker        2550 non-null   object
 7   name                2550 non-null   object
 8   num_speaker         2550 non-null   int64 
 9   published_date      2550 non-null   int64 
 10  ratings             2550 non-null   object
 11  related_talks       2550 non-null   object
 12  speaker_occupation  2544 non-null   object
 13  tags                2550 non-null   object
 14  title               2550 non-null   object
 15  url                 2550 non-null   object
 16  views               2550

In [ ]:
ted_df.shape

(2550, 17)

In [ ]:
ted_df = ted_df['title'].astype(str)
ted_df.head()

0        Do schools kill creativity?
1        Averting the climate crisis
2                   Simplicity sells
3                Greening the ghetto
4    The best stats you've ever seen
Name: title, dtype: object

Text Data Processing Pipeline

In [ ]:
# remove urls, handles, and the hashtag from hashtags
def remove_urls(text):
    new_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return new_text

# make all text lowercase
def text_lowercase(text):
    return text.lower()

# remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text

# remove stopwords
stop_words = set(stopwords.words('english'))


def remove_stopwords(text):

    text = [i
            for i
            in text
            if
            not i
            in stop_words]
    return text

# lemmatize
lemmatizer = WordNetLemmatizer()

def lemmatize(text):

    text = [lemmatizer.lemmatize(token) for token in text]
    return text

In [ ]:
def preprocessing(text):

    text = text_lowercase(text)
    text = remove_urls(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize(text)
    text = remove_stopwords(text)
    text = lemmatize(text)
    text = ' '.join(text)

    return text

In [ ]:
path = '/content/drive/MyDrive/DAtasets/ted_main.csv'

ted_df = pd.read_csv(path)#, quoting=csv.QUOTE_NONE, sep=';', on_bad_lines='skip', encoding=None)
ted_df.head(3)

ted_df.columns

Index(['comments', 'description', 'duration', 'event', 'film_date',
       'languages', 'main_speaker', 'name', 'num_speaker', 'published_date',
       'ratings', 'related_talks', 'speaker_occupation', 'tags', 'title',
       'url', 'views'],
      dtype='object')

# State-of-the-Art Embeddings with GloVe

In [ ]:
#preprocessing input
import nltk
nltk.download('omw-1.4')


for i in range(ted_df.shape[0]):
    ted_df['description'][i] = preprocessing(str(ted_df['description'][i]))

#in case if description has next line character
for text in ted_df.description:
    text=text.replace('\n',' ')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


Generate a feature vector

In [ ]:
#Implementations of above methods
#GloVe:
#loading pre-trained glove model
#downloading and unzipping all word embeddings
!export LC_ALL=en_US.UTF-8

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip
!ls !pwd



--2023-03-12 17:25:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-12 17:25:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-12 17:25:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
#importing 100-d glove model

glove_model_100vec = pd.read_table("glove.6B.100d.txt",
                                   sep=" ",
                                   index_col = 0,
                                   header = None,
                                   quoting=csv.QUOTE_NONE)

In [ ]:
# Getting mean/average vector for each sentence

def get_mean_vector(glove_model, words):
    # remove out-of-vocabulary words
    # assuming 100-d vector

    words = [word
             for word
             in word_tokenize(words)
             if word
             in list(glove_model_100vec.index)] #if word is in vocab

    if len(words) >= 1:
        return np.mean(glove_model_100vec.loc[words].values, axis=0)

    else:
        return np.array([0]*100)


In [ ]:
# creating empty list and appending all mean arrays for comparing cosine
# similarities

glove_vec = []

for i in ted_df.description:

    glove_vec.append(list(get_mean_vector(glove_model_100vec, i)))


glove_vec = np.asarray(glove_vec)
glove_vec

array([[-0.11690753,  0.17445151,  0.04606778, ..., -0.48718723,
         0.28744267,  0.16625453],
       [-0.12658561,  0.17125735,  0.44709804, ..., -0.18936391,
         0.51547109,  0.2958283 ],
       [-0.06018609,  0.12372995,  0.27105957, ..., -0.38565426,
         0.39135596,  0.2519755 ],
       ...,
       [-0.03596823,  0.15682215,  0.13904158, ..., -0.46336053,
         0.47181896,  0.11201082],
       [ 0.06819966,  0.22896074,  0.23709707, ..., -0.27244963,
         0.17535124,  0.22864833],
       [-0.00839048,  0.2554091 ,  0.35424354, ..., -0.30055352,
         0.44468763,  0.16909846]])

# State-of-the-Art Embeddings with ELMo

ELMo vectors are the vectors that are the function of a given sentence. The main advantage of this method is it can have different vectors of words under different contexts.

Words with different contexts in different sentences are called polysemous words. ELMo can successfully handle words of this nature

In [ ]:
!curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.sh -o Miniconda3-latest-MacOSX-x86_64.shbash Miniconda3-latest-MacOSX-x86_64.sh
!conda create --name tf python=3.9
!pip install --upgrade pip
!pip install tensorflow
!python3 -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.9M  100 42.9M    0     0   138M      0 --:--:-- --:--:-- --:--:--  138M
curl: (6) Could not resolve host: Miniconda3-latest-MacOSX-x86_64.sh
/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-03-12 18:27:46.454818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 18:27:48.003822: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not loa

In [ ]:
#Load pre-trained model
import tensorflow as tf
import tensorflow_hub as hub

tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

embed_ = hub.Module("https://tfhub.dev/google/elmo/3")



In [ ]:
# function to average word vectors of each sentence

def elmo_vectors_sentence(x):

  sentence_embeddings = embed_(x.tolist(),
                               signature = "default",
                              as_dict = True)["elmo"]

  with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    sess.run(tf.tables_initializer())

    # Average of each vector
    return sess.run(tf.reduce_mean(sentence_embeddings,1))

#if your data set is large , make a batch of 100 samples. Just remove comment and run the code given below. As we have just 100 samples, we are not doing this
 #samples= [df[i:i+100] for i in range(0,df.shape[0],100)]
 # elmo_vec = [elmo_vectors_sentence(x['description']) for x in samples]
 #elmo_vec_full= np.concatenate(elmo_vec, axis = 0)
#embeddings on our dataset
elmo_vec = elmo_vectors_sentence(ted_df['description'])
elmo_vec


sess = tf.compat.v1.Session()
